<a href="https://colab.research.google.com/github/larissa-mbh/colab-projects/blob/main/IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BERT: Data Preprocessing

In [ ]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q -U tf-models-official==2.7.0
!pip install -U tfds-nightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

In [ ]:
!pip install tensorflow-text
!pip install tensorflow-addons
!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd

tf.get_logger().setLevel('ERROR')

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
def getCSVFromDrive(sharingLink):
  path = 'https://drive.google.com/uc?export=download&id='+sharingLink.split('/')[-2]
  return pd.read_csv(path, sep=";")

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/IC/PAN_style_change

test_data_target = pd.read_csv('./test_target.csv', sep=";")
validation_data_target = pd.read_csv('./validation_target.csv', sep=";")
training_data_target = pd.read_csv('./train_target.csv', sep=";")

test_data_text = pd.read_csv('./test_text.csv', sep=";")
validation_data_text = pd.read_csv('./validation_text.csv', sep=";")
training_data_text = pd.read_csv('./train_text.csv', sep=";")
'''

test_data_target = getCSVFromDrive('https://drive.google.com/file/d/1rEMRzvrk8VS4e_WyLIe2-19yaNKQHbnm/view?usp=share_link')
validation_data_target = getCSVFromDrive('https://drive.google.com/file/d/1-It6bjVH-fzUFP7TYscN1RWUPbGLjux8/view?usp=share_link')
training_data_target = getCSVFromDrive('https://drive.google.com/file/d/1-5jH9nmunpWVYttT2fYb_sOGJY5MDxWx/view?usp=share_link')

test_data_text = getCSVFromDrive('https://drive.google.com/file/d/1-56fRScr4X9FjB6dBV21F79T_o77ykjA/view?usp=share_link')
validation_data_text = getCSVFromDrive('https://drive.google.com/file/d/1-RPmnbQdFpR6xcv33HOZEUgBXFO65YgA/view?usp=share_link')
training_data_text = getCSVFromDrive('https://drive.google.com/file/d/1-Njzm6tJ_i60Eg8CtupMl4-cZOgFbEaJ/view?usp=share_link')

In [ ]:
training_data_text

,text,file-name
0,As stephelton said in the comments to your que...,./train/problem-1.txt
1,I would suspect the OS drive first or whatever...,./train/problem-2.txt
2,The answer lies in SPI - Stateful Packet Inspe...,./train/problem-3.txt
3,Indexing is incredibly resource intensive espe...,./train/problem-4.txt
4,"Do the split just once, what about someone sen...",./train/problem-5.txt
...,...,...
11195,When I load this data into my data warehouse I...,./train/problem-11196.txt
11196,"This is an attribute called origin, which may ...",./train/problem-11197.txt
11197,The problem is that the backup files for the t...,./train/problem-11198.txt
11198,There's another thing which you can do to incr...,./train/problem-11199.txt


In [ ]:
training_data_target

,authors,site,multi-author,file-name
0,3,gamedev.stackexchange.com.7z,1,./train/truth-problem-1.json
1,2,superuser.com.7z,1,./train/truth-problem-2.json
2,3,superuser.com.7z,1,./train/truth-problem-3.json
3,1,superuser.com.7z,0,./train/truth-problem-4.json
4,2,codereview.stackexchange.com.7z,1,./train/truth-problem-5.json
...,...,...,...,...
11195,1,dba.stackexchange.com.7z,0,./train/truth-problem-11196.json
11196,4,networkengineering.stackexchange.com.7z,1,./train/truth-problem-11197.json
11197,4,serverfault.com.7z,1,./train/truth-problem-11198.json
11198,2,codereview.stackexchange.com.7z,1,./train/truth-problem-11199.json


In [ ]:
# subtrair para o número de autores ficar entre [0, 3] para não dar nan no loss
training_data_target['authors'] -= 1
validation_data_target['authors'] -= 1
test_data_target['authors'] -= 1

In [ ]:
training_data_target

,authors,site,multi-author,file-name
0,2,gamedev.stackexchange.com.7z,1,./train/truth-problem-1.json
1,1,superuser.com.7z,1,./train/truth-problem-2.json
2,2,superuser.com.7z,1,./train/truth-problem-3.json
3,0,superuser.com.7z,0,./train/truth-problem-4.json
4,1,codereview.stackexchange.com.7z,1,./train/truth-problem-5.json
...,...,...,...,...
11195,0,dba.stackexchange.com.7z,0,./train/truth-problem-11196.json
11196,3,networkengineering.stackexchange.com.7z,1,./train/truth-problem-11197.json
11197,3,serverfault.com.7z,1,./train/truth-problem-11198.json
11198,1,codereview.stackexchange.com.7z,1,./train/truth-problem-11199.json


In [ ]:
print('Executing eagerly? ', tf.executing_eagerly(), '\nTensorFlow version: ', tf.__version__)

test = {
      'sentence': tf.cast(test_data_text['text'].values, tf.string),
      'label': tf.cast(test_data_target['authors'].values, tf.int8)
    }
validation = {
      'sentence': tf.cast(validation_data_text['text'].values, tf.string),
      'label': tf.cast(validation_data_target['authors'].values, tf.int8)
    }
training = {
      'sentence': tf.cast(training_data_text['text'].values, tf.string),
      'label': tf.cast(training_data_target['authors'].values, tf.int8)
    }
features = ['sentence']
test_sliceDataset = tf.data.Dataset.from_tensor_slices(test)
validation_sliceDataset = tf.data.Dataset.from_tensor_slices(validation)
training_sliceDataset = tf.data.Dataset.from_tensor_slices(training)

print(len(list(test_sliceDataset)), ' ', len(list(validation_sliceDataset)), ' ', len(list(training_sliceDataset)))

Executing eagerly?  True 
TensorFlow version:  2.8.4
2400   2400   11200


In [ ]:
'''
import os

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')
'''

"\nimport os\n\nif os.environ['COLAB_TPU_ADDR']:\n  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')\n  tf.config.experimental_connect_to_cluster(cluster_resolver)\n  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)\n  strategy = tf.distribute.TPUStrategy(cluster_resolver)\n  print('Using TPU')\nelif tf.config.list_physical_devices('GPU'):\n  strategy = tf.distribute.MirroredStrategy()\n  print('Using GPU')\nelse:\n  raise ValueError('Running on CPU is not recommended.')\n"

In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/3',
    'bert_en_wwm_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_en_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/3',
    'bert_en_wwm_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_wwm_cased_L-24_H-1024_A-16/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'albert_en_large':
        'https://tfhub.dev/tensorflow/albert_en_large/2',
    'albert_en_xlarge':
        'https://tfhub.dev/tensorflow/albert_en_xlarge/2',
    'albert_en_xxlarge':
        'https://tfhub.dev/tensorflow/albert_en_xxlarge/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
    'talking-heads_large':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_large/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_wwm_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'bert_en_cased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'bert_en_wwm_uncased_L-24_H-1024_A-16':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'albert_en_large':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'albert_en_xlarge':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'albert_en_xxlarge':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_large':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print('BERT model selected           :', tfhub_handle_encoder)
print('Preprocessing model auto-selected:', tfhub_handle_preprocess)

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocessing model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
def make_bert_preprocess_model(sentence_features, seq_length=128):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load(tfhub_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return tf.keras.Model(input_segments, model_inputs)

In [ ]:
def build_classifier_model(num_classes):

  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
'''
test_classifier_model = build_classifier_model(2)
bert_raw_result = test_classifier_model(text_preprocessed)
print(tf.sigmoid(bert_raw_result))#tf.Tensor([[0.81701595 0.2619492 ]], shape=(1, 2), dtype=float32)
'''

'\ntest_classifier_model = build_classifier_model(2)\nbert_raw_result = test_classifier_model(text_preprocessed)\nprint(tf.sigmoid(bert_raw_result))#tf.Tensor([[0.81701595 0.2619492 ]], shape=(1, 2), dtype=float32)\n'

In [ ]:
tfds_name = 'glue/cola' 

sentence_features = ['sentence']
num_examples = len(list(test_sliceDataset)) + len(list(validation_sliceDataset)) + len(list(training_sliceDataset))

train_split = 'train'
validation_split = 'validation'
test_split = 'test'

target = 'authors'
num_classes = training_data_target[target].nunique()

print(f'This dataset has {num_examples} examples')
print(f'Number of classes: {num_classes}')
print(f'Features {sentence_features}')

# The code below is just to show some samples from the selected dataset
labels_names = ['1 autor', '2 autores', '3 autores', '4 autores']

sample_i = 1
sample_dataset = training_sliceDataset
for sample_row in sample_dataset.take(5):
  samples = [sample_row[feature] for feature in sentence_features]
  print(f'sample row {sample_i}')
  for sample in samples:
    print(sample.numpy())
  sample_label = sample_row['label']

  print(f'label: {sample_label} ({labels_names[sample_label]})')
  print()
  sample_i += 1

This dataset has 16000 examples
Number of classes: 4
Features ['sentence']
sample row 1
b'As stephelton said in the comments to your question, vector math is extremely important for pretty much any 2D or 3D game. However, physics knowledge isn\'t necessary for a lot of simple games. There are physics-like concepts you should understand a bit about, like collision, but you won\'t need calculus or physics classes for that as long as you keep it simple. A lot of things you may want to do can be simulated simply enough that players won\'t care much, like friction, or sliding, or gravity. A decent grasp of physics will likely help in many situations though.\nIt is probably not required to know physics in details when you\'re doing a game, but it definitely helps, especially if there are some \'virtual reality\' features in your game. A game like "From Dust" (Eric Chahi) is essentially physics simulation gamified, while "Another World" only need high-precision capture of real-life motion (so

In [ ]:
len(list(sample_dataset))

11200

In [ ]:
def get_configuration(number_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)

  return metrics, loss

## Treinar

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def load_dataset(dataset, split, batch_size, bert_preprocess_model):
  is_training = split.startswith('train')

  num_examples = len(list(dataset))
  
  if is_training:
    dataset = dataset.shuffle(num_examples)
    dataset = dataset.repeat()
  
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex['label']))
  dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
  print(dataset)
  print(num_examples)

  return dataset, num_examples

In [ ]:
epochs = 6
init_lr = 2e-5

print(f'Fine tuning {tfhub_handle_encoder} model')
bert_preprocess_model = make_bert_preprocess_model(sentence_features)

# USAR GPU
strategy = tf.distribute.MirroredStrategy()


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:
with strategy.scope():

  # metric have to be created inside the strategy scope
  metrics, loss = get_configuration(num_classes)

  batch_size = 32
  train_dataset, train_data_size = load_dataset(training_sliceDataset, train_split, batch_size, bert_preprocess_model)
  validation_dataset, validation_data_size = load_dataset(validation_sliceDataset, validation_split, batch_size,
      bert_preprocess_model)
  test_dataset, test_data_size = load_dataset(test_sliceDataset, test_split, batch_size, bert_preprocess_model)
  
  # train
  steps_per_epoch = train_data_size // batch_size
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = num_train_steps // 10

  # validation
  validation_steps = validation_data_size // batch_size

  # model
  classifier_model = build_classifier_model(num_classes)

  optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

  classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

  history = classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

<PrefetchDataset element_spec=({'input_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'input_word_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'input_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int8, name=None))>
11200
<PrefetchDataset element_spec=({'input_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'input_word_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'input_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int8, name=None))>
2400
<PrefetchDataset element_spec=({'input_mask': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'input_word_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), 'input_type_ids': TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int8, name=None))>
2400
Epoch 1/6
350/350 [==============================] 

In [ ]:
history.history

{'loss': [1.3955930471420288,
  1.3025087118148804,
  1.2088592052459717,
  1.0696592330932617,
  0.9252219796180725,
  0.8221767544746399],
 'accuracy': [0.29973214864730835,
  0.370803564786911,
  0.43267858028411865,
  0.5143749713897705,
  0.5930356979370117,
  0.6407142877578735],
 'val_loss': [1.3113456964492798,
  1.290034294128418,
  1.2901822328567505,
  1.4322125911712646,
  1.550603985786438,
  1.6833090782165527],
 'val_accuracy': [0.3720833361148834,
  0.39125001430511475,
  0.3854166567325592,
  0.3841666579246521,
  0.3570833206176758,
  0.35749998688697815]}

In [ ]:
with strategy.scope():
  print("Evaluate on test data")
  results = classifier_model.evaluate(test_dataset)
  print("test loss, test acc:", results)

Evaluate on test data
75/75 [==============================] - 24s 277ms/step - loss: 1.6570 - accuracy: 0.3762
test loss, test acc: [1.6570461988449097, 0.3762499988079071]


## Exportar

In [ ]:
main_save_path = './my_models'
bert_type = tfhub_handle_encoder.split('/')[-2]
saved_model_name = f'{tfds_name.replace("/", "_")}_{bert_type}'

saved_model_path = os.path.join(main_save_path, saved_model_name)

preprocess_inputs = bert_preprocess_model.inputs
bert_encoder_inputs = bert_preprocess_model(preprocess_inputs)
bert_outputs = classifier_model(bert_encoder_inputs)
model_for_export = tf.keras.Model(preprocess_inputs, bert_outputs)
# Save everything on the Colab host (even the variables from TPU memory)
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model_for_export.save(saved_model_path, include_optimizer=False,
                      options=save_options)


In [ ]:
print('Saving', saved_model_path)
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/IC/PAN_style_change/Models'
saved_model_path = path  + '/model_test'
# Save everything on the Colab host (even the variables from TPU memory)
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model_for_export.save(saved_model_path, include_optimizer=False, options=save_options)
#model_for_export.save(path + '/model_test')

Saving ./my_models/glue_cola_bert_en_uncased_L-12_H-768_A-12
Mounted at /content/drive


In [ ]:
!pip install -U keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.4 requires keras<2.9,>=2.8.0rc0, but you have keras 2.12.0 which is incompatible.


In [ ]:
with tf.device('/job:localhost'):
  reloaded_model = tf.saved_model.load(saved_model_path)

In [ ]:
#reloaded_model = tf.keras.models.load_model(saved_model_path)

## Testar

In [ ]:
def prepare(record):
  model_inputs = [[record[ft]] for ft in sentence_features]
  return model_inputs


def prepare_serving(record):
  model_inputs = {ft: record[ft] for ft in sentence_features}
  return model_inputs


def print_bert_results(test, bert_result, dataset_name):

  bert_result_class = tf.argmax(bert_result, axis=1)[0]

  if dataset_name == 'glue/cola':
    print('sentence:', test[0].numpy())
    if bert_result_class == 1:
      print('This sentence is acceptable')
    else:
      print('This sentence is unacceptable')

  elif dataset_name == 'glue/sst2':
    print('sentence:', test[0])
    if bert_result_class == 1:
      print('This sentence has POSITIVE sentiment')
    else:
      print('This sentence has NEGATIVE sentiment')

  elif dataset_name == 'glue/mrpc':
    print('sentence1:', test[0])
    print('sentence2:', test[1])
    if bert_result_class == 1:
      print('Are a paraphrase')
    else:
      print('Are NOT a paraphrase')

  elif dataset_name == 'glue/qqp':
    print('question1:', test[0])
    print('question2:', test[1])
    if bert_result_class == 1:
      print('Questions are similar')
    else:
      print('Questions are NOT similar')

  elif dataset_name == 'glue/mnli':
    print('premise   :', test[0])
    print('hypothesis:', test[1])
    if bert_result_class == 1:
      print('This premise is NEUTRAL to the hypothesis')
    elif bert_result_class == 2:
      print('This premise CONTRADICTS the hypothesis')
    else:
      print('This premise ENTAILS the hypothesis')

  elif dataset_name == 'glue/qnli':
    print('question:', test[0])
    print('sentence:', test[1])
    if bert_result_class == 1:
      print('The question is NOT answerable by the sentence')
    else:
      print('The question is answerable by the sentence')

  elif dataset_name == 'glue/rte':
    print('sentence1:', test[0])
    print('sentence2:', test[1])
    if bert_result_class == 1:
      print('Sentence1 DOES NOT entails sentence2')
    else:
      print('Sentence1 entails sentence2')

  elif dataset_name == 'glue/wnli':
    print('sentence1:', test[0])
    print('sentence2:', test[1])
    if bert_result_class == 1:
      print('Sentence1 DOES NOT entails sentence2')
    else:
      print('Sentence1 entails sentence2')

  print('BERT raw results:', bert_result[0])
  print()

In [ ]:
def print_bert_results_IC(test, bert_result, dataset_name):

  bert_result_class = tf.argmax(bert_result, axis=1)[0]

  print('Dataset ', dataset_name)
  print('sentence:', test[0].numpy())
  if bert_result_class == 0:
    print('Um autor')
  else:
    print('Múltiplos autores')

  print('BERT raw results:', bert_result[0])
  print()

In [ ]:
with tf.device('/job:localhost'):
  for test_row in test_sliceDataset.shuffle(1000).map(prepare).take(5):
    if len(sentence_features) == 1:
      result = reloaded_model(test_row[0])
    else:
      result = reloaded_model(list(test_row))

    print_bert_results_IC(test_row, result, tfds_name)

Dataset  glue/cola
sentence: [b"Surprisingly, if you do this correctly and responsibly, most ISPs will cooperate with you. They won't identify the user to you, but they will deal with the user.\nYou also might want to look into banning the IPs of free, publicly available proxy servers and prohibit signing up with a list of known free webmail providers to cut down on his options, but even then, a cheap account with a VPN provider and/or an obscure webmail provider can bypass those.\nDoing all of the above is somewhat effective, but a determined troll will get through those blocks, and in my experience, the only thing that really works is moderating new user sign ups, and that causes issues with driving off legitimate new users.  You could just keep banning him until he tires and finds something else to capture his interest (like a shiny object).\nYou can use cookies, Flash cookies or browser fingerprinting to try to remember him no matter his IP, but those are fairly easy to bypass.  A 

In [ ]:
with tf.device('/job:localhost'):
  serving_model = reloaded_model.signatures['serving_default']
  for test_row in test_sliceDataset.shuffle(1000).map(prepare_serving).take(5):
    result = serving_model(**test_row)
    # The 'prediction' key is the classifier's defined model name.
    print_bert_results_IC(list(test_row.values()), result['prediction'], tfds_name)

Dataset  glue/cola
sentence: b'I want to use an AMD Ryzen 2400g, but I will be using OpenCL on three discrete AMD Radeon RX580s installed in it. Will the Vega compute units on the APU still be available if I install the Radeons? Or does their installation disable the integrated vega GPU?\nthere is a limit of 1 gpu however, so if your plan was to use it for mining, likely the cost would just not be worth the attempt, better off to get something like a Ryzen 1200 and a full size x370 or something like that (more pci-e slots)\nRyzen 2200/2400g with a x8 link speed is basically enough bandwidth leftover to feed any current single gpu from AMD or Nvidia up to about Titan V or possibly some dual gpu on 1 card configurations.....they really should have given the ability to disable the Vega graphics core to shift between the iGP and dGPU as needed, maybe this will come with BIOS updates, however, like I said and I have been spending quite a few hours researching this for my own information, yo